In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import time

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import IPython.display as ipd

from scipy.io import wavfile as wav

from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import tensorflow as tf
from tensorflow.keras import regularizers
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping,ModelCheckpoint,LearningRateScheduler, Callback

import plotly.graph_objects as go

In [20]:
def Alice(plaintext_length, fc_layers = 1, activation = "sigmoid"):
    N = plaintext_length
    plaintext_in = Input((N, ), name = "plaintext_input")
    key_in = Input((N, ), name = "key_input")
    x = concatenate([plaintext_in, key_in])
    for i in range(fc_layers):
        x = Dense(2 * N, activation = activation)(x)
    x = tf.expand_dims(x, -1)
    x = Conv1D(2, 4, strides = 1, padding = "same", activation = activation)(x)
    x = Conv1D(4, 2, strides = 2, padding = "same", activation = activation)(x)
    x = Conv1D(4, 1, strides = 1, padding = "same", activation = activation)(x)
    x = Conv1D(1, 1, strides = 1, padding = "same", activation = "tanh")(x)
    outputs = tf.squeeze(x, axis = -1)
    model = keras.Model(inputs = [plaintext_in, key_in], outputs = outputs)
    return model

def Bob(ciphertext_length, fc_layers = 1, activation = "sigmoid"):
    N = ciphertext_length
    ciphertext_in = Input((N, ), name = "ciphertext_input")
    key_in = Input((N, ), name = "key_input")
    x = concatenate([ciphertext_in, key_in])
    for i in range(fc_layers):
        x = Dense(2 * N, activation = activation)(x)
    
    x = tf.expand_dims(x, -1)
    x = Conv1D(2, 4, strides = 1, padding = "same", activation = activation)(x)
    x = Conv1D(4, 2, strides = 2, padding = "same", activation = activation)(x)
    x = Conv1D(4, 1, strides = 1, padding = "same", activation = activation)(x)
    x = Conv1D(1, 1, strides = 1, padding = "same", activation = "tanh")(x)
    outputs = tf.squeeze(x, axis = -1)
    model = keras.Model(inputs = [plaintext_in, key_in], outputs = outputs)
    return model

def Eve(plaintext_length, fc_layers = 1, activation = "sigmoid"):
    N = plaintext_length
    ciphertext_in = Input((N, ), name = "ciphertext_input")
    
    x = ciphertext_in
    for i in range(fc_layers):
        x = Dense(2 * N, activation = activation)(x)
        
    x = tf.expand_dims(x, axis = -1)
    x = Conv1D(2, 4, strides = 1, padding = "same", activation = activation)(x)
    x = Conv1D(4, 2, strides = 2, padding = "same", activation = activation)(x)
    x = Conv1D(4, 1, strides = 1, padding = "same", activation = activation)(x)
    x = Conv1D(1, 1, strides = 1, padding = "same", activation = "tanh")(x)
    outputs = tf.squeeze(x, axis = -1)
    model = keras.Model(inputs = [plaintext_in, key_in], outputs = outputs)
    return model


In [ ]:
L1_loss = keras.losses.MeanAbsoluteError()

def alice_bob_loss(y, y_pred, N, eve_loss):
    L1_comp = L1_loss(y, y_pred)
    eve_comp = (N**2 - tf.square(eve_loss)) / (N**2)
    
    return L1_comp + eve_comp

def train(plaintext, key, alice, bob, eve, learning_rate):
    

In [ ]:
def run_experiment()